# Task 1

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt
import json

In [ ]:
#Importing Feature Geometry Shapefiles
catchments_primary = gpd.read_file("Catchments/catchments_primary.shp")
catchments_secondary = gpd.read_file("Catchments/catchments_secondary.shp")
catchments_future = gpd.read_file("Catchments/catchments_future.shp")
sa = gpd.read_file("SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp")
unemployment = gpd.read_file("Data/SA2_unemployment.shp")

In [ ]:
#Importing csv, txt files
businesses=pd.read_csv("Businesses.csv")
income=pd.read_csv("Income.csv")
polling=pd.read_csv("PollingPlaces2019.csv")
population=pd.read_csv("Population.csv")
schools = pd.read_json("NSW_Schools_JSON.json")
toilets = pd.read_csv('Australia Public Toilet Map.csv')

stops=pd.read_csv("Stops.txt", sep=",")

In [ ]:
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
import json

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        try:
            db = create_engine('postgresql+psycopg2://'+db_user+':'+db_pw+'@'+host+'/'+default_db, echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(sqlcmd, args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

In [ ]:
db, conn = pgconnect(credentials)

Connected successfully.


In [ ]:
unemployment

,SA2_MAIN,SA2_MAIN16,SA2_NAME,STATE_CODE,STATE_NAME,AREA_SQKM,PER_UNEMPL,geometry
0,101021007,101021007,Braidwood,1,New South Wales,3418.3525,4.2,"POLYGON ((149.56837 -36.00876, 149.56813 -36.0..."
1,101021008,101021008,Karabar,1,New South Wales,6.9825,4.7,"POLYGON ((149.23877 -35.36226, 149.23867 -35.3..."
2,101021009,101021009,Queanbeyan,1,New South Wales,4.7634,5.2,"POLYGON ((149.23877 -35.36226, 149.23764 -35.3..."
3,101021010,101021010,Queanbeyan - East,1,New South Wales,13.0034,5.0,"POLYGON ((149.24148 -35.33667, 149.24308 -35.3..."
4,101021011,101021011,Queanbeyan Region,1,New South Wales,3054.4099,2.9,"POLYGON ((149.76065 -35.08348, 149.75962 -35.0..."
...,...,...,...,...,...,...,...,...
571,128021537,128021537,Royal National Park,1,New South Wales,139.3336,0.0,"POLYGON ((151.07257 -34.04744, 151.07293 -34.0..."
572,128021538,128021538,Sutherland - Kirrawee,1,New South Wales,7.7547,4.0,"POLYGON ((151.04088 -34.02826, 151.04163 -34.0..."
573,128021607,128021607,Engadine,1,New South Wales,8.9538,3.0,"POLYGON ((151.04044 -34.03608, 151.04041 -34.0..."
574,128021608,128021608,Loftus - Yarrawarrah,1,New South Wales,3.8436,3.0,"POLYGON ((151.04255 -34.03530, 151.04289 -34.0..."


In [28]:
with open("live-traffic-cameras.json") as f:
    data = json.load(f)
    
data

[{'geo_point_2d': {'lon': 151.18022, 'lat': -33.86789},
  'region': 'SYD_MET',
  'title': 'Anzac Bridge (Eastbound)',
  'view': 'Intersection of Victoria Road and Anzac Bridge looking east towards the Sydney CBD.',
  'direction': 'E',
  'href': 'https://roads-waterways.transport.nsw.gov.au/trafficreports/cameras/camera_images/anzacbr.jpg',
  'photo': {'thumbnail': True,
   'filename': 'anzacbr.jpg',
   'width': 1024,
   'format': 'JPEG',
   'etag': '"640ac732-227cf"',
   'mimetype': 'image/jpeg',
   'id': '3d2c3a9e5c64ab0e623eed998a5a1d1e',
   'last_synchronized': '2023-03-10T06:00:05.457194',
   'color_summary': ['rgba(108, 113, 121, 1.00)',
    'rgba(95, 92, 89, 1.00)',
    'rgba(104, 99, 99, 1.00)'],
   'height': 768}},
 {'geo_point_2d': {'lon': 151.00533, 'lat': -33.80866},
  'region': 'SYD_WEST',
  'title': 'Victoria Road (Parramatta)',
  'view': 'Victoria Road at Church Street looking east towards Rydalmere.',
  'direction': 'E',
  'href': 'https://roads-waterways.transport.nsw.g

In [29]:
df = pd.DataFrame(columns=["geo_point_2d", "region", "title", "view", "direction", "href", "photo"])

for i in range(0, len(data)):
    currentItem = data[i]
    df.loc[i] = [data[i]["geo_point_2d"], data[i]["region"], data[i]["title"], data[i]["view"], data[i]["direction"], data[i]["href"], data[i]["photo"]]


In [30]:
df[['lon', 'lat']] = df["geo_point_2d"].apply(pd.Series)
df

,geo_point_2d,region,title,view,direction,href,photo,lon,lat
0,"{'lon': 151.18022, 'lat': -33.86789}",SYD_MET,Anzac Bridge (Eastbound),Intersection of Victoria Road and Anzac Bridge...,E,https://roads-waterways.transport.nsw.gov.au/t...,"{'thumbnail': True, 'filename': 'anzacbr.jpg',...",151.180220,-33.867890
1,"{'lon': 151.00533, 'lat': -33.80866}",SYD_WEST,Victoria Road (Parramatta),Victoria Road at Church Street looking east to...,E,https://roads-waterways.transport.nsw.gov.au/t...,"{'thumbnail': True, 'filename': 'churchst_parr...",151.005330,-33.808660
2,"{'lon': 151.05237, 'lat': -33.77967}",SYD_NORTH,Cumberland Highway (Carlingford),Cumberland Highway at Marsden Road looking nor...,N,https://roads-waterways.transport.nsw.gov.au/t...,"{'thumbnail': True, 'filename': 'cumberlandhwy...",151.052370,-33.779670
3,"{'lon': 151.20595, 'lat': -33.45451}",REG_NORTH,M1 Pacific Motorway (Mount White),M1 Pacific Motorway at the Mount White heavy v...,N,https://roads-waterways.transport.nsw.gov.au/t...,"{'thumbnail': True, 'filename': 'f3_mountwhite...",151.205950,-33.454510
4,"{'lon': 151.20084, 'lat': -33.82768}",SYD_NORTH,Falcon Street (Crows Nest),Corner of Falcon Street and the Pacific Highwa...,E,https://roads-waterways.transport.nsw.gov.au/t...,"{'thumbnail': True, 'filename': 'falconst_crow...",151.200840,-33.827680
...,...,...,...,...,...,...,...,...,...
159,"{'lon': 150.978459, 'lat': -33.386823}",SYD_NORTH,Webbs Creek ferry (West),St Albans Road at Webbs Creek ferry looking we...,W,https://roads-waterways.transport.nsw.gov.au/t...,"{'thumbnail': True, 'filename': 'stalbans_cobb...",150.978459,-33.386823
160,"{'lon': 150.876052, 'lat': -33.502874}",SYD_WEST,Sackville ferry (South),Sackville Road at Sackville ferry looking nort...,N,https://roads-waterways.transport.nsw.gov.au/t...,"{'thumbnail': True, 'filename': 'sackville_tiz...",150.876052,-33.502874
161,"{'lon': 149.591139, 'lat': -33.418282}",SYD_WEST,Great Western Highway (Bathurst),Great Western Highway at Havannah Street looki...,E,https://roads-waterways.transport.nsw.gov.au/t...,"{'thumbnail': True, 'filename': 'GWH_Havannah....",149.591139,-33.418282
162,"{'lon': 150.885556, 'lat': -33.438139}",SYD_WEST,Lower Portland Ferry (West),West Portland Road at Lower Portland Ferry loo...,W,https://roads-waterways.transport.nsw.gov.au/t...,"{'thumbnail': True, 'filename': 'LowerPortland...",150.885556,-33.438139
